### 11.3.1生成候选项集
---
#### 一些辅助函数
数据集扫描函数的伪代码如下：
```python
对数据集中的每条交易记录tran:
    对每个候选项集can:
        if can是tran的子集：
            增加can的计数值
对每个候选项集：
    if 其支持度>=最小支持度：
        则保留该项集
返回所有频繁项集列表
```

#### 程序清单11-1 Apriori算法中的辅助函数

In [6]:
def loadDataSet():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]


def createC1(dataSet):
    """生成候选项集"""
    c1=[]
    for transaction in dataSet:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()    # 这个.sort()有啥意思，元素是列表，怎么排序
    return list(map(frozenset,c1))    # frozenset不可更改的集合


def scanD(D,ck,minSupport):
    """基于最小支持度,筛选出频繁项集
    D:交易记录
    ck:候选项集
    """
    # 计算每个候选项集的个数
    ssCnt={}    # 键是项集，值是次数
    for tid in D:    # 对每个交易记录（set化了的）
        for can in ck:    # 对每个候选项集
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can]=1
                else:
                    ssCnt[can]+=1
    numItems=float(len(D))
    retList=[]
    supportData={}
    # 计算所有项集的支持度
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support>=minSupport:
            retList.insert(0,key)
        supportData[key]=support
    return retList,supportData

In [3]:
# 函数测试
dataSet=loadDataSet()
c1=createC1(dataSet)
c1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [4]:
# 集合形式的data:
D=list(map(set,dataSet,))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [7]:
L1,supportData0=scanD(D,c1,0.5)    # 利用最小支持度过滤频繁项集
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

#### 函数测试没问题
---
### 11.3.2 组织完整的Apriori算法
#### 伪代码
```python
当集合中项的个数大于0时：
    构建一个k个项组成的候选项集的列表
    检查数据以确认每个项集都是频繁的
    保留频繁项集并构建 k+1 项组成的候选项集的列表
```
---
#### 程序清单11-2 Apriori算法

In [9]:
def aprioriGen(Lk,k):
    """辅助函数,用于生成Ck,生成每个项集有k个元素的项集列表
    Lk:元素为集合的列表
    k:要生成的每个集合的长度。
    """
    retList=[]
    lenLk=len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1=list(Lk[i])[:k-2]
            L2=list(Lk[j])[:k-2]
            L1.sort()    # 对列表排序，保证元素从小到大排序
            L2.sort()
            if L1==L2:
                retList.append(Lk[i]|Lk[j])
    return retList


def apriori(dataSet,minSupport=0.5):
    """apriori算法主函数"""
    c1=createC1(dataSet)    # 候选项集
    D=list(map(set,dataSet))    # 将每条交易记录集合化
    L1,supportData=scanD(D,c1,minSupport)    # 基于支持度得到频繁项集，（及其对应的支持度）
    L=[L1]
    k=2    # 初始化k
    
    while len(L[k-2])>0:    # L中会不断的加入更大的项集，直到下一个更大的项集为空，while退出
        Ck=aprioriGen(L[k-2],k)
        Lk,supK=scanD(D,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k+=1
    return L,supportData


apriori(dataSet)    # 注意到L[3]为空列表，其长度为0，所以其导致while退出

([[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
  [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
  [frozenset({2, 3, 5})],
  []],
 {frozenset({1}): 0.5,
  frozenset({3}): 0.75,
  frozenset({4}): 0.25,
  frozenset({2}): 0.75,
  frozenset({5}): 0.75,
  frozenset({1, 3}): 0.5,
  frozenset({2, 5}): 0.75,
  frozenset({3, 5}): 0.5,
  frozenset({2, 3}): 0.5,
  frozenset({1, 5}): 0.25,
  frozenset({1, 2}): 0.25,
  frozenset({2, 3, 5}): 0.5})

## 11.4 从频繁项集中挖掘出关联规则
---
频繁项集对应支持度，而关联规则对应可信度
如规则P->H的可信度定义为：
$$\frac{support(P|H)}{support(P)}$$
##### 其中，P|H是指所有出现在集合P或集合H中的元素
---
#### 程序清单11-3 关联规则生成函数

In [ ]:
# 两个辅助函数
from typing import List


def calcConf(freqSet,H,supportData,br1:List,minConf=0.7):
    """计算可信度
    返回所有满足最小置信度要求的规则列表
    """
    prunedH=[]
    for conseq in H:
        conf=supportData[freqSet]/supportData[freqSet-conseq]
        if conf>=minConf:
            print(freqSet-conseq,'--->',conseq,'conf: ',conf)
            br1.append((freqSet-conseq,conseq,conf))    # 对br1做原地增加操作。
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet,H,supportData,br1:List,minConf=0.7):
    """尝试进一步合并"""
    